In [8]:
import networkx as nx
import numpy as np
from dimod import DiscreteQuadraticModel
#from dwave.system import LeapHybridDQMSampler

In [20]:
# Modularization with DQM solver



def modularization(G,B,num_partitions):

    partitions = range(num_partitions)
    # Initialize the DQM object
    dqm = DiscreteQuadraticModel()


    for i in G.nodes:
        dqm.add_variable(num_partitions, label=i)


    for i in G.nodes: 
        linear_term = np.ones(num_partitions)*B[i,i]*(-1)
        dqm.set_linear(i, linear_term)

    for i,j in G.edges:
        dqm.set_quadratic(i,j, {(c, c): ((-1)*B[i,j]) for c in partitions})

    # Initialize the DQM solver
    sampler = LeapHybridDQMSampler()
   # sampler = greedy.SteepestDescentSampler()
    #sampleset = sampler.sample(dqm)

    # Solve the problem using the DQM solver
    sampleset = sampler.sample_dqm(dqm)

    # get the first solution
    sample = sampleset.first.sample
    energy = sampleset.first.energy

    # Count the nodes in each partition
    counts = np.zeros(num_partitions)

    for i in sample:
        counts[sample[i]] += 1

    # Compute the number of links between different partitions
    sum_diff = 0
    for i, j in G.edges:
        if sampleset.first.sample[i] != sampleset.first.sample[j]:
            sum_diff += 1
    
    return (sample, energy, counts, sum_diff)

In [28]:
# Number of nodes in the graph
num_nodes = 30
# Create a random geometric graph
G = nx.random_geometric_graph(n=num_nodes, radius=0.4, dim=2, seed=518)



def all_modularization(G):
    B = nx.modularity_matrix(G, nodelist=sorted(G.nodes()), weight='weight')
    all_solutions = []
    
    for num_partitions in range (2, int(np.sqrt(len(G))+1),1):  #Wurzel in Literatur oft als passend angeführt, e.g.Yuanyuan 2018 (QEA) 
    
        result=modularization(G,B,num_partitions)
        all_solutions.append({'number_of_partitions': num_partitions, 'solution': result[0], 'energy':result[1]})
    
        if (num_partitions==2):
            best_energy=result[1]
            best_solution=result[0]
        else:
            if result[1] < best_energy:
                best_energy = result[1]
                best_solution=result[0]
    
    
    
    return(best_solution, best_energy)